In [ ]:
!pip install pandas_ta > /tmp/install.log 2>&1 || exit
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas_ta as pta
import numpy as np
import math
import re
from sklearn.preprocessing import StandardScaler

In [ ]:
%%capture
!mkdir -p content || exit
![ -d content/stock-market-analysis ] && (rm -r content/stock-market-analysis || exit)
outpt=!wget https://stream24wall.web.app/stock-market-dataset/shares.zip --directory-prefix=content/stock-market-analysis/datasets || exit;

In [ ]:
%%capture
outpt=!unzip -o content/stock-market-analysis/datasets/shares.zip -d content/stock-market-analysis/datasets/ || exit;

In [ ]:
class Visualisation:
    @staticmethod
    def show_scaled_plot(data_input, title, show_legend=False):
        data_pivot = data_input.pivot(index='DATE', columns='ISIN', values='PRICE')
        df_scaled = pd.DataFrame(StandardScaler().fit_transform(data_pivot), index=data_pivot.index,
                                 columns=data_pivot.columns)

        ax = df_scaled.plot(legend=False)
        if show_legend:
            ax.legend(bbox_to_anchor=(1.1, 1.05))
        plt.title(title)
        plt.ylabel("Price")
        plt.xlabel("Date")
        plt.gcf().autofmt_xdate()
        plt.show()

    @staticmethod
    def show_trades_plot(ax, stock_data, stock_trade_data, stock_ta_data, isin):
        stock_data = stock_data[stock_data['ISIN'].isin([isin])]
        stock_trade_data = stock_trade_data[stock_trade_data['ISIN'].isin([isin])].copy()
        stock_ta_data = stock_ta_data[stock_ta_data['ISIN'].isin([isin])].copy()
        stock_trade_data['BUYS'] = np.where(stock_trade_data['ACTION'] == 'BUY', 1, 0)
        stock_trade_data['SELLS'] = np.where(stock_trade_data['ACTION'] == 'SELL', -1, 0)
        stock_data = pd.merge(stock_data,
                              stock_trade_data,
                              how='left',
                              on=['ISIN', 'DATE'])
        aggregated_stock_data = stock_data[['DATE', 'BUYS', 'SELLS']].groupby(['DATE']).sum()

        ax2 = ax.twinx()
        ax2.set_ylim([-2, 2])
        ax2.bar(aggregated_stock_data.index, aggregated_stock_data['BUYS'] * 0.1, width=1, color='g', bottom=-1.5)
        ax2.bar(aggregated_stock_data.index, aggregated_stock_data['SELLS'] * 0.1, width=1, color='r', bottom=-1.5)
        ax2.set_yticks([])

        ax3 = ax.twinx()
        ax3.plot(stock_ta_data['DATE'], stock_ta_data['RSI_14'], color='m')
        ax3.grid(False)
        ax3.set_ylim([100,200])
        ax3.set_yticks([10, 50, 90])
        # ax3.set_ylabel('RSI')

        ax.plot(stock_data['DATE'], stock_data['PRICE'], color='b')
        ax.grid(True)
        plt.title(re.findall(r'name=([^,]+)|$', stock_trade_data.iloc[0]['DATA'])[0] + ", " + isin, pad=0)
        ax.set_zorder(1)
        ax.patch.set_visible(False)
        plt.setp(ax.get_xticklabels(), rotation=15, ha='right')

In [ ]:
stockLookupData = pd.read_csv("content/stock-market-analysis/datasets/share_lookup.csv",
                              index_col="ID")

stockPriceData = pd.read_csv("content/stock-market-analysis/datasets/share.csv",
                        parse_dates=['TS'])

stockTradeData = pd.read_csv("content/stock-market-analysis/datasets/share_trade.csv",
                             parse_dates=['TS'])

# Use last 33 days only
stockPriceData = stockPriceData[stockPriceData['TS'] >= pd.Timestamp.today() - pd.offsets.Day(33)]
stockTradeData = stockTradeData[stockTradeData['TS'] >= pd.Timestamp.today() - pd.offsets.Day(33)]

stockPriceData = pd.merge(stockPriceData,
                     stockLookupData,
                     on='ID')

stockPriceData['DATE'] = stockPriceData['TS'].dt.date
stockPriceData = stockPriceData.drop(['ID', 'TS'], axis=1)
stockPriceData = stockPriceData.groupby(['DATE', 'ISIN'])['PRICE'].mean().reset_index(level=['DATE', 'ISIN'])

stockTradeData['DATE'] = stockTradeData['TS'].dt.date
stockTradeData = stockTradeData.drop(['TS', 'MS'], axis=1)

**Scaled Stock Prices**

In [ ]:
Visualisation.show_scaled_plot(stockPriceData, "Scaled Stock Prices")

**Top 25 Oversold Stocks, RSI Heatmap**

In [ ]:
CustomStrategy = pta.Strategy(
    name="TA Idicators",
    ta=[
        {"kind": "rsi", "length": 14, "close": "PRICE"}
    ]
)

def apply_strat(param):
    param.ta.strategy(CustomStrategy)
    return param

dfTaIndicatorsData = stockPriceData.groupby(['ISIN'], group_keys=False).filter(lambda x: len(x) >= 14).groupby(['ISIN'], group_keys=False).apply(apply_strat)

dfTopRsi = dfTaIndicatorsData.groupby(['DATE', 'ISIN'])[['RSI_14']].max().sort_values(by=['DATE', 'RSI_14'], ascending=[False, True]).head(25).reset_index(level=0)
dfFlopRsi = dfTaIndicatorsData.groupby(['DATE', 'ISIN'])[['RSI_14']].max().sort_values(by=['DATE', 'RSI_14'], ascending=[False, False]).head(25).reset_index(level=0)
dfTaIndicators = dfTaIndicatorsData[dfTaIndicatorsData['ISIN'].isin(dfTopRsi.index)]
dfTaIndicatorsPivot = dfTaIndicators.pivot_table(index='DATE', columns='ISIN', values='RSI_14')

fig = plt.figure(figsize=(40, 15))
plt.title("Top 25 Oversold Stocks, RSI Heatmap")
sns.heatmap(dfTaIndicatorsPivot[dfTopRsi.index], cmap='RdYlBu_r')
plt.show()
dfTopRsi

**Top 10 Oversold Stocks, Scaled Prices**

In [ ]:
Visualisation.show_scaled_plot(stockPriceData[stockPriceData['ISIN'].isin(dfTopRsi.head(10).index)], "Top 10 Oversold Stocks, Scaled Prices", True)

**Next 10 Oversold Stocks, Scaled Prices**

In [ ]:
Visualisation.show_scaled_plot(stockPriceData[stockPriceData['ISIN'].isin(dfTopRsi.tail(15).head(10).index)], "Next 10 Oversold Stocks, Scaled Prices", True)

**Top 10 Overbought Stocks, Scaled Prices**

In [ ]:
Visualisation.show_scaled_plot(stockPriceData[stockPriceData['ISIN'].isin(dfFlopRsi.head(10).index)], "Top 10 Overbought Stocks, Scaled Prices", True)

**Stock Trades Charts with Buys, Sells and RSI**

In [ ]:
stockTradeDataIsins = stockTradeData['ISIN'].drop_duplicates()
fig = plt.figure()
fig.set_figwidth(15)
fig.set_figheight(5 * stockTradeDataIsins.count() / 2)
idx = 1
for isin in stockTradeDataIsins:
    ax = plt.subplot(math.ceil(stockTradeDataIsins.count() / 2), 2, idx)
    Visualisation.show_trades_plot(ax, stockPriceData, stockTradeData, dfTaIndicatorsData, isin)
    idx += 1
plt.show()

Disclaimer: Educational Purposes Only

The financial and International Securities Identification Number (ISIN) data listed on this platform is provided solely for educational purposes. The information is intended to serve as general guidance and does not constitute financial advice, an endorsement, or a recommendation for the purchase or sale of any securities.

While we strive to ensure the accuracy and timeliness of the information presented, we make no representations or warranties, express or implied, regarding the completeness, accuracy, reliability, suitability, or availability of the provided data. Users are encouraged to independently verify any information obtained from this platform before making any investment decisions.

This platform and its operators are not responsible for any errors, omissions, or inaccuracies in the provided data, nor for any actions taken in reliance on such information. Users are strongly advised to conduct thorough research and seek the advice of qualified financial professionals before making any investment decisions.

The use of International Securities Identification Numbers (ISINs) and other financial data is subject to various regulations and licensing agreements. Users are responsible for complying with all applicable laws and respecting any terms and conditions associated with the use of such data.

By accessing and using this platform, users acknowledge and agree that they are doing so at their own risk and discretion. This educational content is not a substitute for professional financial advice, and users should consult with qualified professionals for specific guidance tailored to their individual circumstances.